In [ ]:
from nbdev import *
%nbdev_hide_input
!date

Sun Feb 28 09:07:22 PST 2021


# Geminga Analyses

> Ongoing analyses


** `wtlike` library functions used here **
- `get_cells`
- `simulate_cells`
- `get_lightcurve`
- `get_bb_partition`
- `partition_cells`
- `fit_cells`
- `flux_plot`


## Introduction

This notebook does not produce a module to add to the library, it a place to run, and report on analyses using the library. It uses local code, shown below.

This is also an exploration of this style of presenting code, and data analysis based on it.

## Geminga analyses

Following Kerr's example, we use the source Geminga to verify that this analysis of data gives results consistent with its being constant. As you can see below, the BB analysis finds several breaks, which merit further checks. That is followed by a simulation using
Geminga's weight distribution and exposure. The run shown here finds one minor break, a change of 0.08%. 

In [ ]:
%nbdev_collapse_input
from light_curves.tools import *
from wtlike.config import *
from utilities.ipynb_docgen import *
config = Config()
source = PointSource('geminga')

def show_cache():
    """
    ## Current Cache
    {the_cache}
    """
    import pickle, datetime
    sname = source.name
    with capture_print('contents') as the_cache:                    
        c = config.cache
        s = f'Cache contents for source {sname}\n  {"key":20}  {"size":>10}  {"time":20}\n'
        for name,value in c.items():
            if name.endswith(sname):
                stat = value.stat()
                size = stat.st_size
                mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
                s += f'  {name:20s}  {size:10}  {mtime:20} \n'
        print(s)
    return locals()

if config.valid:
    nbdoc(show_cache)
else:
    print('Invalid data--not generating plots.')

## Current Cache
<details  class="nbdoc-description" >  <summary> contents </summary>  <div style="margin-left: 5%"><pre>Cache contents for source geminga<br>  key                         size  time                <br>  photons_geminga         31679265  2021-02-28 08:20     <br>  exposure_geminga        86263776  2021-02-28 08:22     <br>  binexp_geminga             64448  2021-02-28 08:22     <br>  cells_geminga            5132223  2021-02-28 08:22     <br>  lightfcurve_geminga      7289195  2021-02-28 08:39     <br>  BB-geminga                   281  2021-02-28 08:43     <br>  weight_hist_geminga          556  2021-02-28 08:43     <br><br></pre></div> </details>


In [ ]:
%nbdev_collapse_input
def geminga_combined():
    """
    ## Fit to all data
    {outp}
    This is the likelihood, for the combined dataset, a consistency check of the weight calculation.
    {fig1}
    """
       
    with capture_print('Analysis output') as outp:
        ll = all_data_likelihood(config, source)

    #print(ll,'\nFit: ', ll.fit_info())
    fig1, ax1 = plt.subplots(num=1, figsize=(3,2))
    ll.plot(xlim =( 0.99, 1.01), ax=ax1)
    return locals()
if config.valid:
    nbdoc(geminga_combined)

## Fit to all data
<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Cell data for geminga: Restoring from cache with key "cells_geminga"<br></pre></div> </details>
This is the likelihood, for the combined dataset, a consistency check of the weight calculation.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/geminga_combined_fig_01.png" title="images/geminga_combined_fig_01.png">    <img src="images/geminga_combined_fig_01.png" alt="Figure 1 at images/geminga_combined_fig_01.png" >   </a> </figure>
</div>



In [ ]:
%nbdev_collapse_input
def analysis_plots(name, expect=0.9963, short=50, simname=''):
    """
    ### {sim} Geminga data
    
    {which_source} the daily binned data, or cells; then perform a Bayesian Blocks partition;
    finally make fits to the resulting blocks. <br>(Run at {date})
    
    {output}
    
    This shows the fits to all cells, with the BB fit overlays.
    {fig1}
    
    Since this is a constant source, there should be no breaks, that is, only one partition.
    Here is a table of the partition fits:
    
    {df_text}
    The last column represents the compatibility of the flux measurement for each partition
    with the expected value {expect} in equivalent sigma units.
    
    Expand the plot around short, < {short} day partitions.
    {short_check}
    
    {fig2}
    """

    simulated = bool(simname)
    sim= 'Simulated' if simulated else ''
    which_source = 'Simulate a set of ' if simulated else 'Get'
    with capture_print('Analysis output' ) as output:
        if not simulated:
            lc, bb_lc = analyze_data(config, source)
        else: 
            lc, bb_lc = simulation(config, source, bb_key=simname) 

    pd.set_option('display.precision', 3)#, 'display.colheader_justify','left')
    
    df = fit_table(bb_lc, expect=expect)
    df_text = monospace(str(df), 'BB fit table', open=True)
        
    plt.rc('font', size=16)
    fig1, ax = plt.subplots(1,1, sharex=True, figsize=(10,4), num=1)
    bb_overplot(config, lc, bb_lc, ax = ax)
    ax.text(0.05, 0.85, name,  transform=ax.transAxes);
    fig1.width=600

    bb_short = bb_lc.query(f'tw<{short}'); ns =len(bb_short)
    if ns>0:
        short_check=f'There are {ns} such.'
        rows, cols = (ns+2)//3, 3
        fig2, axx = plt.subplots( rows,3,  figsize=(5*cols, 4*rows),
                                 sharey=True, sharex=True,
                     gridspec_kw=dict(top=0.85, left=0.08, bottom=0.15, hspace=0.2 ),num=2)
        if ns>1: fig2.width=600
        axf = axx.flatten()
        [ax.set_visible(False) for ax in axf[ns:]]
        for t, ax in zip(bb_short.t, axf):
            bb_overplot(config, lc, bb_lc, ax=ax, tzero=t, xlim=(-50, +50))
    else:
        fig2=''
        short_check = 'None found.'
    return locals()

In [ ]:
%nbdev_collapse_input
if config.valid:
    nbdoc(analysis_plots, 'Geminga')

###  Geminga data

Get the daily binned data, or cells; then perform a Bayesian Blocks partition;
finally make fits to the resulting blocks. <br>(Run at 2021-02-28 09:07)

<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Light curve for geminga: Restoring from cache with key "lightfcurve_geminga"<br>Cell data for geminga: Restoring from cache with key "cells_geminga"<br>BB-geminga: Restoring from cache with key "BB-geminga"<br>Partitioned 3873 cells into 15 blocks, using LikelihoodFitness <br>Loaded 15 / 15 cells with exposure &gt; 0.3 for fitting<br></pre></div> </details>

This shows the fits to all cells, with the BB fit overlays.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/analysis_plots_fig_01.png" title="images/analysis_plots_fig_01.png">    <img src="images/analysis_plots_fig_01.png" alt="Figure 1 at images/analysis_plots_fig_01.png" width=600>   </a> </figure>
</div>


Since this is a constant source, there should be no breaks, that is, only one partition.
Here is a table of the partition fits:

<details open class="nbdoc-description" >  <summary> BB fit table </summary>  <div style="margin-left: 5%"><pre>          t      tw       n   flux           errors  sigma_dev  limit<br>0   55027.0   688.0  226266  1.002  (-0.003, 0.003)        1.9  1.007<br>1   55378.0    14.0    4175  1.127  (-0.024, 0.025)        5.5  1.168<br>2   55559.0   348.0  104103  0.988  (-0.004, 0.004)       -1.9  0.995<br>3   55746.5    27.0    8423  1.119  (-0.017, 0.017)        7.3  1.147<br>4   56114.0   708.0  208558  0.995  (-0.003, 0.003)       -0.5  1.000<br>5   56487.0    38.0   11673  1.082  (-0.014, 0.014)        6.3  1.106<br>6   56512.5    13.0    3701  0.905  (-0.021, 0.022)       -4.1  0.941<br>7   56674.0   310.0   91733  0.993  (-0.005, 0.005)       -0.8  1.000<br>8   56880.0   102.0   30315  1.030  (-0.008, 0.008)        4.0  1.043<br>9   56972.5    83.0   22865  0.969  (-0.009, 0.009)       -2.9  0.985<br>10  57017.5     7.0    2259  1.161  (-0.034, 0.034)        5.0  1.218<br>11  57109.0   176.0   49643  1.007  (-0.006, 0.006)        1.6  1.017<br>12  57197.5     1.0     157  0.536   (-0.067, 0.07)       -5.7  0.658<br>13  57917.0  1438.0  401432  0.998  (-0.002, 0.002)        0.7  1.002<br>14  58666.5    61.0   11453  1.083  (-0.014, 0.014)        6.1  1.107</pre></div> </details>
The last column represents the compatibility of the flux measurement for each partition
with the expected value 0.9963 in equivalent sigma units.

Expand the plot around short, < 50 day partitions.
There are 6 such.

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/analysis_plots_fig_02.png" title="images/analysis_plots_fig_02.png">    <img src="images/analysis_plots_fig_02.png" alt="Figure 2 at images/analysis_plots_fig_02.png" width=600>   </a> </figure>
</div>



In [ ]:
%nbdev_collapse_input
if config.valid:

    nbdoc(analysis_plots, 'Geminga', name='analysis_sim',  simname='analysis_plot_sim', 
         expect=0.9915)

### Simulated Geminga data

Simulate a set of  the daily binned data, or cells; then perform a Bayesian Blocks partition;
finally make fits to the resulting blocks. <br>(Run at 2021-02-28 09:07)

<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Light curve for geminga: Restoring from cache with key "lightfcurve_geminga"<br>Cell data for geminga: Restoring from cache with key "cells_geminga"<br>binned exposure for source geminga: Restoring from cache with key "binexp_geminga"<br>Weight histogram for geminga: Restoring from cache with key "weight_hist_geminga"<br>Loaded 3873 / 4015 cells with exposure &gt; 0.3 for fitting<br>analysis_plot_sim: Restoring from cache with key "analysis_plot_sim"<br>Partitioned 3873 cells into 2 blocks, using LikelihoodFitness <br>Loaded 2 / 2 cells with exposure &gt; 0.3 for fitting<br></pre></div> </details>

This shows the fits to all cells, with the BB fit overlays.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/analysis_sim_fig_01.png" title="images/analysis_sim_fig_01.png">    <img src="images/analysis_sim_fig_01.png" alt="Figure 1 at images/analysis_sim_fig_01.png" width=600>   </a> </figure>
</div>


Since this is a constant source, there should be no breaks, that is, only one partition.
Here is a table of the partition fits:

<details open class="nbdoc-description" >  <summary> BB fit table </summary>  <div style="margin-left: 5%"><pre>         t      tw        n   flux           errors  sigma_dev  limit<br>0  56318.0  3270.0  1031971  0.992  (-0.001, 0.001)        0.1  0.994<br>1  58325.0   744.0   186592  0.995  (-0.003, 0.003)        0.9  1.000</pre></div> </details>
The last column represents the compatibility of the flux measurement for each partition
with the expected value 0.9915 in equivalent sigma units.

Expand the plot around short, < 50 day partitions.
None found.




# TO DO
- Fix a 0.4% bias from the cell fit when applied to high-statisics data.

- Look at the intervals detected for the data, of which 5 have measured flux increases around 10%,  more than 4$\sigma$. Possibilities are a problem with the exposure, and a change in the background. The latter can be examined by a 2-D fit with $\beta$ free. Another possibility, fixing $\alpha=0$ and fitting $\beta$ is not (yet) supported.
